In [0]:
raw_posts_df = spark.read.table('data_platform.default.raw_posts')

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import DataFrame

def split_tags_into_array(df: DataFrame) -> DataFrame:
    return(
        df.withColumn('TagsArray', F.filter(F.split(F.col('Tags'), r'\|'), lambda x: x!= '')).drop('Tags')
    )

def rename_columns(df: DataFrame) -> DataFrame:
    return(
        df.withColumnRenamed('Id', 'PostId')
    )

def map_post_type(df: DataFrame) -> DataFrame:
    map_data = [
        (1, "Question"),
        (2, "Answer"),
        (3, "Orphaned tag wiki"),
        (4, "Tag wiki excerpt"),
        (5, "Tag wiki"),
        (6, "Moderator nomination"),
        (7, "Wiki placeholder"),
        (8, "Privilege wiki"),
        (9, "Article"),
        (10, "HelpArticle"),
        (12, "Collection"),
        (13, "ModeratorQuestionnaireResponse"),
        (14, "Announcement"),
        (15, "CollectiveDiscussion"),
        (17, "CollectiveCollection")
    ]
    map_schema = StructType([
        StructField('PostTypeId', IntegerType(), False),
        StructField('PostType', StringType(), False)
    ])
    map_df = spark.createDataFrame(map_data, schema = map_schema)

    return(
        df.join(
            F.broadcast(map_df),
            on = 'PostTypeId',
            how = 'left'
        )
    )

stg_posts_df = (
    raw_posts_df
    .transform(split_tags_into_array)
    .transform(rename_columns)
    .transform(map_post_type)
)





In [0]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from delta.tables import DeltaTable


def incremental_upsert(dest_table: str, df: DataFrame, unique_key: str, updated_at: str, full_refresh = False):
    if not spark.catalog.tableExists(dest_table) or full_refresh:
        (
            df.write
                .format('delta')
                .mode('overwrite')
                .option('overwriteSchema', 'true')
                .saveAsTable(dest_table)
        )
    else:
        latest_date = (
            spark.read.table(dest_table)
                .agg(F.max(updated_at).alias('max_ts'))
                .collect()[0]['max_ts']
        )
        increment_df = df.filter(F.col(updated_at) > latest_date)
        if increment_df.limit(1).count() > 0:
            delta_table = DeltaTable.forName(spark, dest_table)
            (
                delta_table.alias('d').merge(
                    source = increment_df.alias('i'),
                    condition = f'd.{unique_key} = i.{unique_key}'
                )
                .whenMatchedUpdateAll()
                .whenNotMatchedInsertAll()
                .execute()
            )
dest_table = 'data_platform.default.stg_posts'
incremental_upsert(dest_table, stg_posts_df, 'PostId', 'CreationDate')

In [0]:
num_partitions = len(spark.table(dest_table).inputFiles())
display(num_partitions)
